<a href="https://colab.research.google.com/github/mauricionoris/fm/blob/master/atv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 2 — Distâncias e Similaridades (Colab)

**Conteúdo prático**  
1) Produto escalar  
2) Comprimento (módulo)  
3) Similaridade angular (cosseno)  
4) Distâncias: Euclidiana, Manhattan e Jaccard  

**Foco em IA**: terminologia aplicada a **embeddings**, **busca semântica**, **k-NN/k-means**, e **recomendação baseada em conjuntos**.


## 0) Setup rápido
- Usaremos apenas `numpy` e algumas utilitárias.
- Para conjuntos/binários (Jaccard), mostraremos duas formas: com `set` e com vetores {0,1}.


In [1]:
# @title Imports
import numpy as np
from typing import Iterable, Tuple
np.set_printoptions(suppress=True, precision=4)


## 1) Produto escalar (dot product)
Interpretar como **quanto um vetor aponta na direção do outro**. Muito usado para medir **afinidade** entre **embeddings** (texto, imagem, áudio).


In [2]:
# @title Produto escalar (função e exemplo)
def dot(x: np.ndarray, y: np.ndarray) -> float:
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    return float(np.dot(x, y))

# Exemplo simples: dois embeddings 2D (apenas para visualização)
x = np.array([3, 4])
y = np.array([1, 2])
print('x·y =', dot(x, y))


x·y = 11.0


## 2) Comprimento (módulo)
O **tamanho** do vetor. Em IA, ajuda a entender **intensidade/magnitude** de um embedding.


In [3]:
# @title Comprimento (função e exemplo)
def length(v: np.ndarray) -> float:
    v = np.asarray(v, dtype=float)
    return float(np.linalg.norm(v))

print('||x|| =', length(x))
print('||y|| =', length(y))


||x|| = 5.0
||y|| = 2.23606797749979


## 3) Similaridade angular (cosseno) e ângulo


- Similaridade do cosseno $$
\operatorname{sim}_{\cos}(x,y)=\frac{x\cdot y}{\|x\|\,\|y\|}
$$
- Ângulo:  $$
\theta=\cos^{-1}\!\left(\frac{x\cdot y}{\|x\|\,\|y\|}\right)
$$

Em embeddings, **direção** importa mais que magnitude — por isso a similaridade angular é muito usada em **busca semântica** e **rankings de proximidade**.


In [4]:
# @title Similaridade do cosseno (funções e exemplo)
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        raise ValueError('cosine: vetores não podem ter comprimento zero')
    return float(np.dot(a, b) / (na * nb))

def angle_from_cosine_sim(sim: float) -> float:
    # Ângulo em radianos
    sim = max(-1.0, min(1.0, sim))
    return float(np.arccos(sim))

sim = cosine_similarity(x, y)
theta = angle_from_cosine_sim(sim)
print('cosine(x,y) =', sim)
print('ângulo(x,y) [rad] =', theta)


cosine(x,y) = 0.9838699100999074
ângulo(x,y) [rad] = 0.17985349979247847


## 4) Distâncias — Euclidiana, Manhattan e Jaccard
- **Euclidiana**: “reta” entre pontos (boa para **k-means** e **k-NN** com dados bem escalados).  
- **Manhattan**: soma das diferenças por coordenada (robusta a **outliers** / dados esparsos).  
- **Jaccard**: dissimilaridade entre **conjuntos** (ou vetores binários), ótima para **recomendação** por itens vistos/curtidos.


In [5]:
# @title Distâncias (funções) + Demonstração
def dist_euclid(a: np.ndarray, b: np.ndarray) -> float:
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    return float(np.linalg.norm(a - b))

def dist_manhattan(a: np.ndarray, b: np.ndarray) -> float:
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    return float(np.abs(a - b).sum())

def jaccard_sets(A: Iterable, B: Iterable) -> float:
    A, B = set(A), set(B)
    if not A and not B:
        return 0.0  # distância 0 entre conjuntos vazios
    inter = len(A & B)
    union = len(A | B)
    return float(1 - inter / union)

def jaccard_binary(a: np.ndarray, b: np.ndarray) -> float:
    # a, b são vetores binários {0,1}
    a = np.asarray(a).astype(int)
    b = np.asarray(b).astype(int)
    assert a.shape == b.shape
    inter = int(np.logical_and(a==1, b==1).sum())
    union = int(np.logical_or (a==1, b==1).sum())
    if union == 0:
        return 0.0
    return float(1 - inter / union)

# Demonstração
x3 = np.array([2, -1, 0], dtype=float)
y3 = np.array([1,  1, 2], dtype=float)
print('Euclidiana(x3,y3) =', dist_euclid(x3, y3))
print('Manhattan (x3,y3) =', dist_manhattan(x3, y3))

A = {'item1','item2','item3','item4'}
B = {'item2','item3','item5'}
print('Jaccard (sets A,B) =', jaccard_sets(A, B))

xb = np.array([1,0,1,1,0,0,1])
yb = np.array([0,1,1,0,0,1,1])
print('Jaccard (binário)  =', jaccard_binary(xb, yb))


Euclidiana(x3,y3) = 3.0
Manhattan (x3,y3) = 5.0
Jaccard (sets A,B) = 0.6
Jaccard (binário)  = 0.6666666666666667


## 5) Mini-caso de uso (embeddings e recomendação)
- Temos 4 *embeddings* de documentos/itens em 3D.  
- Consulta: achamos os **mais similares (angular)** e **mais próximos (euclidiana/manhattan)**.  
- Para **Jaccard**, simulamos itens consumidos por usuário (conjuntos).


In [7]:
# @title Mini-caso de uso
docs = np.array([
    [0.9, 0.1, 0.1],   # doc A
    [0.85, 0.12, 0.05],# doc B
    [0.1, 0.8, 0.1],   # doc C
    [0.1, 0.1, 0.9],   # doc D
], dtype=float)
labels = ['A','B','C','D']
q = np.array([0.92, 0.05, 0.08])  # consulta

sims   = [cosine_similarity(q, d) for d in docs]
distsE = [dist_euclid(q, d) for d in docs]
distsM = [dist_manhattan(q, d) for d in docs]

print('Consulta q:', q)
print('\nTop-3 por similaridade angular (cosseno):')
for i in np.argsort(sims)[::-1][:3]:
    print(labels[i], 'sim=', round(sims[i],4))

print('\nTop-3 por distância euclidiana:')
for i in np.argsort(distsE)[:3]:
    print(labels[i], 'dist=', round(distsE[i],4))

print('\nTop-3 por distância Manhattan:')
for i in np.argsort(distsM)[:3]:
    print(labels[i], 'dist=', round(distsM[i],4))

# Jaccard com conjuntos (consumo de itens)
U1 = {'a','b','c','d'}
U2 = {'b','c','e'}
U3 = {'a','c','e','f'}
print('\nDistâncias Jaccard entre perfis de consumo (conjuntos):')
print('d_J(U1,U2)=', jaccard_sets(U1, U2))
print('d_J(U1,U3)=', jaccard_sets(U1, U3))
print('d_J(U2,U3)=', jaccard_sets(U2, U3))


Consulta q: [0.92 0.05 0.08]

Top-3 por similaridade angular (cosseno):
A sim= 0.9982
B sim= 0.9959
D sim= 0.2006

Top-3 por distância euclidiana:
A dist= 0.0574
B dist= 0.1034
C dist= 1.1114

Top-3 por distância Manhattan:
A dist= 0.09
B dist= 0.17
C dist= 1.59

Distâncias Jaccard entre perfis de consumo (conjuntos):
d_J(U1,U2)= 0.6
d_J(U1,U3)= 0.6666666666666667
d_J(U2,U3)= 0.6


## 6) 20 Exercícios (terminologia de caso de uso)
Use linguagem de IA: *embedding*, *consulta*, *rank*, *vizinhos*, *perfis de consumo*, *itens*.
Responda no Colab, adicionando células abaixo de cada item conforme precisar.


1. (Dot) Dado `x=[3,4,0]` e `y=[1,2,2]`, calcule `x·y` e interprete o sinal/magnitude no contexto de **afinidade de embeddings**.  
2. (Len) Para `x=[-2,1,2]`, calcule `||x||` e explique como o tamanho do embedding pode afetar métricas sensíveis à magnitude.  
3. (Cos) Para `x=[2,0,0]` e `y=[10,0,0]`, calcule a **similaridade do cosseno** e o **ângulo**; explique por que são semanticamente muito próximos.  
4. (Euc) Calcule a **distância euclidiana** entre `x=[1,2]` e `y=[4,6]` e diga se `y` é um **vizinho próximo** de `x`.  
5. (Man) Calcule a **distância Manhattan** entre `x=[1,2,3]` e `y=[2,4,2]`; quando preferir Manhattan em **dados esparsos**?  
6. (Jac-sets) Para `U1={a,b,c,d}` e `U2={b,c,e}`, calcule `d_J(U1,U2)` e interprete como **dissimilaridade de interesses**.  
7. (Jac-bin) Converta `U1,U2` em vetores binários relativos ao universo `{a,b,c,d,e}` e calcule Jaccard binário; compare com o valor via conjuntos.  
8. (Rank-cos) Dados `D1=[0.9,0.1,0.1]`, `D2=[0.85,0.12,0.05]`, `D3=[0.1,0.8,0.1]` e `q=[0.92,0.05,0.08]`, rankeie por **similaridade angular**.  
9. (Rank-euc) Com os mesmos docs e consulta, rankeie por **distância euclidiana**; compare com o exercício 8.  
10. (Rank-man) Repita para **Manhattan**; discuta coerências/divergências com o *rank* angular.  
11. (Escala) Mostre um exemplo em que multiplicar um embedding por 10 **não** muda o *rank* por cosseno, mas muda por euclidiana.  
12. (Ortogonais) Encontre `x,y` não nulos com `x·y=0` e interprete como **tópicos independentes** em embeddings.  
13. (Ângulo pequeno) Construa `x,y` com `cos≈0.99` e discuta a **semelhança semântica** esperada.  
14. (Conjuntos idênticos) Mostre que para `U={a,b,c}`, `d_J(U,U)=0` (identidade dos indiscerníveis).  
15. (Conjuntos disjuntos) Mostre que se `A∩B=∅`, então `d_J(A,B)=1` (má dissimilaridade).  
16. (Trade-off) Crie `x,y` em 3D onde Euclidiana é pequena mas o ângulo é grande; discuta **magnitude vs direção**.  
17. (k-vizinhos) Implemente função para retornar **k-vizinhos** por cosseno e por euclidiana; compare para `k=2` usando `docs` e `q`.  
18. (Jaccard perfis) Para três usuários `U1,U2,U3` (defina conjuntos diferentes), compute a **matriz de distâncias Jaccard** e identifique o par mais alinhado.  
19. (Perturbação) Perturbe ligeiramente `q` e mostre o efeito no *rank* por cosseno vs euclidiana (robustez a **ruído de escala**).  
20. (Pipeline) Crie uma função que receba **lista de embeddings** e uma **consulta** e retorne top-*k* por (i) cosseno, (ii) euclidiana, (iii) manhattan, e **explique quando usar cada um**.


In [9]:
# @title Utilitárias: top-k e matriz Jaccard
def topk_by_metric(q, X, k=3, metric='cosine') -> Tuple[np.ndarray, np.ndarray]:
    X = np.asarray(X, dtype=float)
    assert metric in {'cosine','euclid','manhattan'}
    if metric=='cosine':
        vals = np.array([cosine_similarity(q, x) for x in X])
        order = np.argsort(vals)[::-1]  # maior é melhor
    elif metric=='euclid':
        vals = np.array([dist_euclid(q, x) for x in X])
        order = np.argsort(vals)        # menor é melhor
    else:
        vals = np.array([dist_manhattan(q, x) for x in X])
        order = np.argsort(vals)        # menor é melhor
    return order[:k], vals[order[:k]]

def jaccard_matrix_sets(users_sets: dict) -> np.ndarray:
    keys = list(users_sets.keys())
    n = len(keys)
    M = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            M[i, j] = jaccard_sets(users_sets[keys[i]], users_sets[keys[j]])
    return M

# Mini-demo
idx, vals = topk_by_metric(q, docs, k=2, metric='cosine')
print('Top-2 (cosine):', [labels[i] for i in idx], vals)
idx, vals = topk_by_metric(q, docs, k=2, metric='euclid')
print('Top-2 (euclid):', [labels[i] for i in idx], vals)


Top-2 (cosine): ['A', 'B'] [0.9982 0.9959]
Top-2 (euclid): ['A', 'B'] [0.0574 0.1034]
